In [1]:
!python --version

Python 3.7.6


In [2]:
!pwd

/root


In [3]:
import magenta

In [4]:
CONFIG = 'basic_rnn'
BUNDLE_PATH = '/root/magenta/magenta/models/melody_rnn/mag/basic_rnn.mag'
OUTPUT_DIR = '/root/magenta/magenta/models/melody_rnn/output/2020.3.7-01'

# 用 pre-trained model 生成序列

In [12]:
!python /root/magenta/magenta/models/melody_rnn/melody_rnn_generate.py \
--config='attention_rnn' \
--bundle_file='/root/magenta/magenta/models/melody_rnn/mag/attention_rnn.mag' \
--output_dir='/root/magenta/magenta/models/melody_rnn/output/2020.3.7-02' \
--num_outputs=5 \
--num_steps=256 \
--primer_midi='/root/magenta/magenta/models/melody_rnn/primer.mid'

2020-03-07 21:37:46.117185: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-03-07 21:37:46.131090: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2494220000 Hz
2020-03-07 21:37:46.131524: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559fcce4b2b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-07 21:37:46.131553: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-07 21:37:46.171534: W tensorflow/core/framework/op_def_util.cc:357] Op TensorArray is deprecated. It will cease to work in GraphDef version 16. Use TensorArrayV3.
2020-03-07 21:37:46.172288: W tensorflow/core/framework/op_def_util.cc:357] Op TensorArrayScatter is deprecated. It will cease to work in GraphDef version 19. Use TensorArrayGradV3.
2020-03-07 21:37:46.173057: W tensor